In [3]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
import networkx as nx
import pandas as pd
import numpy as np
from tqdm import tqdm
import itertools

df=pd.read_feather("../../data/interim/work_school_home_sp_esc.feather")
df['id'] = df.index + 1

In [5]:
def clusterize_relation(df, relation_name='home', x_coord='home_x', y_coord='home_y',
                            new_col_name = 'Neighbourhood', n_participants = 10, seed=13):
    data = []
    means = []
    for zone in df[relation_name].unique():
        if pd.notna(zone):
            tmp = df[df[relation_name] == zone][['id', x_coord, y_coord, relation_name]].copy()
            n_clusters = int(np.sum(len(tmp)) / n_participants)

            if n_clusters < 2:
                tmp[new_col_name] = tmp[relation_name].astype(int).astype(str) + '_'  + '0'

            else: 
                X = tmp[[x_coord, y_coord]]
                kmeans = KMeans(n_clusters=n_clusters, random_state=seed).fit(X)
                tmp[new_col_name] = kmeans.labels_
                tmp[new_col_name] = (tmp[relation_name].astype(int).astype(str) + '_' +
                                     tmp[new_col_name].astype(int).astype(str))
                    
            data.append(tmp)
                
    df = pd.concat(data)

    return df

def make_clusters(df, home_cluster_n=10, work_cluster_n=20, school_cluster_n=30):
    home_clusters = clusterize_relation(df,'home','home_x','home_y','home_cluster',home_cluster_n)
    work_clusters = clusterize_relation(df,'work','work_x','work_y','work_cluster',work_cluster_n)
    school_clusters = clusterize_relation(df,'school','school_x','school_y','school_cluster',school_cluster_n)

    return merge_cluster_dataframes(df, home_clusters, work_clusters, school_clusters)

def merge_cluster_dataframes(df, home_clusters, work_clusters, school_clusters):
    merged = pd.merge(df, home_clusters.drop(['home_x', 'home_y'], axis=1), 
                                                                   on=['id', 'home'], how='left')
    
    merged = pd.merge(merged, work_clusters.drop(['work_x', 'work_y'], axis=1),
                                                                       on=['id', 'work'], how='left')
    
    merged = pd.merge(merged, school_clusters.drop(['school_x', 'school_y'], axis=1), 
                                                                             on=['id', 'school'], how='left')
    
    assert merged[merged['home'].notna()]['home_cluster'].isna().sum() == 0
    assert merged[merged['work'].notna()]['work_cluster'].isna().sum() == 0
    assert merged[merged['school'].notna()]['school_cluster'].isna().sum() == 0
    
    return merged

df = make_clusters(df, 10, 20, 30)

In [13]:
def add_person_to_graph(G, person):
    G.add_node(person['id'],
               work = person['work'],
               school = person['school'],
               home = person['home'],
               age = person['idade'],
               private_healthcare = person['private_healthcare'],
               home_id = person['home_id'],
               education = person['education'],
               home_x= person['home_x'], home_y = person['home_y'], 
               school_x = person['school_x'], school_y = person['school_y'],
               work_x = person['work_x'], work_y = person['work_y'],
               criterio_br = person['criteriobr'] )    

def add_people_to_graph(G, df):
    print('Adding People Nodes')
    df.apply(lambda x: add_person_to_graph(G, x), axis=1)
    print(25*'*')

def add_edge(G, person1, person2, edge_type, relation_cluster, edge_zone):
    G.add_edge(person1, person2, edge_type=edge_type, cluster=relation_cluster, zone=edge_zone)

def add_edges(G, df, relation, cluster, edge_type):
    print(f'Adding {edge_type} Edges')
    total_edges = 0
    for c in tqdm(df[cluster].unique()):
        if pd.notna(c):
            tmp = df[df[cluster] == c]
            assert len(np.unique(tmp[relation])) == 1
            zone = tmp[relation].iloc[0]
            if len(tmp) > 1:
                combinations = list(itertools.combinations(tmp['id'].values, 2))
                total_edges += len(combinations)
                for p1, p2 in combinations:
                    add_edge(G,p1, p2, edge_type, c, zone)
   
    print(len([True for x,y,v in G.edges.data(data=True) if v['edge_type'] == edge_type]))
    print(25*'*')
    return total_edges

In [14]:
G = nx.MultiGraph()
add_people_to_graph(G, df)
houses = add_edges(G, df, 'home', 'home_id', 'home')
neighbors = add_edges(G, df, 'home', 'home_cluster', 'neighbor')
works = add_edges(G, df, 'work',   'work_cluster', 'work')
schools = add_edges(G, df, 'school', 'school_cluster', 'school')
assert len(G.nodes()) == len(df)
assert houses == len([True for x,y,v in G.edges.data(data=True) if v['edge_type'] == 'home'])
assert neighbors == len([True for x,y,v in G.edges.data(data=True) if v['edge_type'] == 'neighbor'])
assert works == len([True for x,y,v in G.edges.data(data=True) if v['edge_type'] == 'work'])
assert schools == len([True for x,y,v in G.edges.data(data=True) if v['edge_type'] == 'school'])


Adding People Nodes


  0%|                                                                              | 31/21708 [00:00<01:11, 304.81it/s]

*************************
Adding home Edges


  1%|▍                                                                              | 31/5401 [00:00<00:17, 301.77it/s]

61634
*************************
Adding neighbor Edges


...

348835
*************************
Adding work Edges


  9%|██████▉                                                                         | 37/423 [00:00<00:02, 188.84it/s]

342525
*************************
Adding school Edges


100%|███████████████████████████████████████████████████████████████████████████████| 423/423 [00:02<00:00, 201.62it/s]


251454
*************************


In [15]:
df

,home,school,work,id,work_x,work_y,school_x,school_y,home_x,home_y,home_id,idade,criteriobr,renda_fa,education,private_healthcare,home_cluster,work_cluster,school_cluster
0,1.0,NaN,3.0,1,333104.0,7394476.0,NaN,NaN,333743.0,7394463.0,00010001,59.0,4.0,2732.575910,3.0,False,1_1,3_0,NaN
1,1.0,84.0,82.0,2,327503.0,7392159.0,329431.0,7395939.0,333743.0,7394463.0,00010001,21.0,4.0,2732.575910,4.0,False,1_1,82_16,84_1
2,1.0,NaN,1.0,3,333453.0,7394501.0,NaN,NaN,333814.0,7394428.0,00010002,37.0,5.0,3200.000000,4.0,False,1_4,1_7,NaN
3,1.0,NaN,1.0,4,333539.0,7394387.0,NaN,NaN,333814.0,7394428.0,00010002,19.0,5.0,3200.000000,3.0,False,1_4,1_7,NaN
4,1.0,NaN,26.0,5,332344.0,7393317.0,NaN,NaN,333814.0,7394428.0,00010002,18.0,5.0,3200.000000,3.0,False,1_4,26_1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55487,342.0,NaN,90.0,55488,327737.0,7396374.0,NaN,NaN,323322.0,7392038.0,03421904,27.0,1.0,9461.630458,5.0,True,342_10,90_0,NaN
55488,342.0,NaN,341.0,55489,323653.0,7391801.0,NaN,NaN,322857.0,7390956.0,03422006,52.0,1.0,9461.630458,5.0,True,342_6,341_0,NaN
55489,342.0,NaN,341.0,55490,323653.0,7391801.0,NaN,NaN,322857.0,7390956.0,03422006,26.0,1.0,9461.630458,5.0,True,342_6,341_0,NaN
55490,342.0,NaN,342.0,55491,323034.0,7390476.0,NaN,NaN,323034.0,7390476.0,03422109,28.0,3.0,3000.000000,5.0,False,342_3,342_1,NaN


In [16]:
df.to_feather("../../data/processed/clusterized_df.feather")

In [17]:
nx.write_gpickle(G, '../../data/processed/SP_multiGraph_intID.gpickle')